In [1]:
!pip install transformers
!pip install sentencepiece
!pip install emoji
!pip install vncorenlp
!pip install sentence_transformers

In [2]:
import torch
import emoji
import re
import gc
import os
import json
import csv
import logging as lg
import pandas as pd
import numpy as np
import math

from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from sklearn.model_selection import StratifiedKFold, KFold
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from torch import nn
from torch.nn import LSTM
from transformers import *
from vncorenlp import VnCoreNLP
from nltk.tokenize import TweetTokenizer
from pandas import DataFrame

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
!nvidia-smi

Sun Dec 12 09:48:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    34W / 250W |   1851MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
train_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/public_train.csv"
val_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/val.csv"
test_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/final_private_test.csv"

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6]:
def normalizeToken(token):
    if len(token) == 1:
        return emoji.demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

In [7]:
def isnan(s):
    return s != s


def normalizePost(post, tweet_tokenizer, vncorenlp, use_segment=False, remove_punc_stopword=False, lowercase_opt=False, truncation_method="head_only", length=512):
    post = post.strip()
    URL_pattern = r"(?:http?s://|www.)[^\"]+"
    hashtag_pattern = r"#\w+"

    post = re.sub(URL_pattern, "", post)
    post = re.sub(hashtag_pattern, "hashtag", post)
    post = re.sub("\.+",".", post)
    #post = re.sub("#\s+", " ", post)
    post = re.sub("\*+", " ", post)
    post = re.sub("\$+", "đô ", post)
    post = re.sub("-{2,}", "", post)
    post = re.sub("\@+", "", post)
    post = re.sub("\[[0-9]?[0-9]]", " : dẫn_chứng ", post)

    post = post.strip()
    if lowercase_opt:
      post = post.lower()
    tokens = tweet_tokenizer.tokenize(post.replace("’", "'").replace("…", "..."))
    
    post = " ".join(tokens)
    if use_segment:
        tokens = vncorenlp.tokenize(post.replace("’", "'").replace("…", "..."))
        tokens = [t for ts in tokens for t in ts]
    normPost = " ".join(tokens)

    if remove_punc_stopword:
      tokens = [t for t in normPost if not t in vnmese_stopwords]
    normPost = " ".join(tokens)

    normPost = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normPost)
    normPost = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normPost)
    normPost = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normPost)
    if use_segment:
        normPost = normPost.replace('< url >', '<url>')
        normPost = re.sub(r"# (\w+)", r'#\1', normPost)
    if truncation_method == "head_only":
      normPost = " ".join(normPost.split(" ")[:length])
    if truncation_method == "tail_only":
      normPost = " ".join(normPost.split(" ")[-length:])
    if truncation_method == "head_tail":
      normPost = " ".join(normPost.split(" ")[:int(length*0.25)]) + " " +  " ".join(normPost.split(" ")[-int(length*0.75):])
    return normPost

In [8]:
data_train = pd.read_csv(train_path)
data_val = pd.read_csv(val_path)
data_test = pd.read_csv(test_path)

In [9]:
#Time stamp feature
data_train['timestamp_post'] = pd.to_numeric(data_train['timestamp_post'], errors='coerce')
data_train['timestamp_post'] = data_train['timestamp_post'].astype('float')

data_test['timestamp_post'] = pd.to_numeric(data_test['timestamp_post'], errors='coerce')
data_test['timestamp_post'] = data_test['timestamp_post'].astype('float')

data_val['timestamp_post'] = pd.to_numeric(data_val['timestamp_post'], errors='coerce')
data_val['timestamp_post'] = data_val['timestamp_post'].astype('float')

#Number of shares feature
data_train['num_share_post'] = pd.to_numeric(data_train['num_share_post'], errors='coerce')
data_train['num_share_post'] = data_train['num_share_post'].astype('float')

data_test['num_share_post'] = pd.to_numeric(data_test['num_share_post'], errors='coerce')
data_test['num_share_post'] = data_test['num_share_post'].astype('float')

data_val['num_share_post'] = pd.to_numeric(data_val['num_share_post'], errors='coerce')
data_val['num_share_post'] = data_val['num_share_post'].astype('float')

#Number of likes feature
data_train['num_like_post'] = pd.to_numeric(data_train['num_like_post'], errors='coerce')
data_train['num_like_post'] = data_train['num_like_post'].astype('float')

data_test['num_like_post'] = pd.to_numeric(data_test['num_like_post'], errors='coerce')
data_test['num_like_post'] = data_test['num_like_post'].astype('float')

data_val['num_like_post'] = pd.to_numeric(data_val['num_like_post'], errors='coerce')
data_val['num_like_post'] = data_val['num_like_post'].astype('float')

#Number of comments feature
data_train['num_comment_post'] = pd.to_numeric(data_train['num_comment_post'], errors='coerce')
data_train['num_comment_post'] = data_train['num_comment_post'].astype('float')

data_test['num_comment_post'] = pd.to_numeric(data_test['num_comment_post'], errors='coerce')
data_test['num_comment_post'] = data_test['num_comment_post'].astype('float')

data_val['num_comment_post'] = pd.to_numeric(data_val['num_comment_post'], errors='coerce')
data_val['num_comment_post'] = data_val['num_comment_post'].astype('float')

data_train['post_message'] = data_train['post_message'].fillna('empty')
data_val['post_message'] = data_val['post_message'].fillna('empty')

In [10]:
class ElectraReINTELClassification(ElectraPreTrainedModel):
    def __init__(self, config):
        super(ElectraReINTELClassification, self).__init__(config=config)
        self.electra = ElectraModel(config)
        self.num_labels = config.num_labels
        self.init_weights()
        self.ln = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask, position_ids=position_ids, head_mask=head_mask)[1]
        cls_output = torch.cat((outputs[-1][:, 0, ...], outputs[-2][:, 0, ...], outputs[-3][:, 0, ...], outputs[-4][:, 0, ...]), -1)
        logits = self.ln(cls_output)
        return logits

In [11]:
class RobertaReINTELClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(RobertaReINTELClassification, self).__init__(config)
        self.roberta = RobertaModel(config)
        self.num_labels = config.num_labels
        self.outputs = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        cls_output = torch.cat((outputs[2][-1][:, 0, ...], outputs[2][-2][:, 0, ...], outputs[2][-3][:, 0, ...], outputs[2][-4][:, 0, ...]), -1)
        logits = self.outputs(cls_output)
        return logits

In [12]:
class BertReINTELClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(BertReINTELClassification, self).__init__(config=config)
        self.bert = BertModel(config)
        self.num_labels = config.num_labels
        self.ln = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, position_ids=position_ids, head_mask=head_mask)[2]

        cls_output = torch.cat((outputs[-1][:, 0, ...], outputs[-2][:, 0, ...], outputs[-3][:, 0, ...], outputs[-4][:, 0, ...]), -1)
        logits = self.ln(cls_output)
        return logits

In [13]:
def null_index_list(df, feature):
  return df[feature][df[feature].isnull()].index.tolist()

def fill_missing_median(df, feature):
  df[feature] = df[feature].fillna(df[feature].median())

In [14]:
nan_timestamp_idxs = null_index_list(data_train, "timestamp_post")
nan_like_idxs = null_index_list(data_train, "num_like_post")
nan_share_idxs = null_index_list(data_train, "num_share_post")
nan_comment_idxs = null_index_list(data_train, "num_comment_post")

val_nan_timestamp_idxs = null_index_list(data_val, "timestamp_post")
val_nan_like_idxs = null_index_list(data_val, "num_like_post")
val_nan_share_idxs = null_index_list(data_val, "num_share_post")
val_nan_comment_idxs = null_index_list(data_val, "num_comment_post")

test_nan_timestamp_idxs = null_index_list(data_test, "timestamp_post")
test_nan_like_idxs = null_index_list(data_test, "num_like_post")
test_nan_share_idxs = null_index_list(data_test, "num_share_post")
test_nan_comment_idxs = null_index_list(data_test, "num_comment_post")

In [15]:
tweet_tokenizer = TweetTokenizer()
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/VLSP-Fake-News-Detection/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg")

train_sents = []
test_sents = []
val_sents = []

# If PhoBERT: use_segment = True, otherwise False
# normalizePost(post, tweet_tokenizer, vncorenlp, use_segment=False, remove_punc_stopword=False, lowercase_opt=False, truncation_method="head_only", length=512)
for post in data_train.post_message:
  train_sents.append(normalizePost(post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

for post in data_val.post_message:
  val_sents.append(normalizePost(post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

for test_post in data_test.post_message:
  test_sents.append(normalizePost(test_post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

In [16]:
from sentence_transformers import SentenceTransformer
sent_model = SentenceTransformer('bert-base-nli-mean-tokens')

train_sen_embeddings = sent_model.encode(train_sents)
test_sen_embeddings = sent_model.encode(test_sents)
val_sen_embeddings = sent_model.encode(val_sents)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
train_pairwise_similarities = cosine_similarity(train_sen_embeddings)
val_pairwise_similarities = cosine_similarity(val_sen_embeddings)
test_pairwise_similarities = cosine_similarity(test_sen_embeddings)

In [18]:
#Fill missing value by copying and pasting their most similar post message's values to them
def fill_missing_similarity(df, pairwise_similarity, feature, features_list):
  for i in features_list:
    max_val, pos_i, pos_j = -1, -1, -1
    for j in range(len(pairwise_similarity[i])):
      if i == j or j in features_list:
        continue
      elif pairwise_similarity[i][j] > max_val:
        #max_val = pairwise_similarities[i][j]
        pos_i = i
        pos_j = j
    df[feature][pos_i] = df[feature][pos_j]

In [19]:
fill_missing_similarity(data_train, train_pairwise_similarities, "timestamp_post", nan_timestamp_idxs)
fill_missing_similarity(data_val, val_pairwise_similarities, "timestamp_post", val_nan_timestamp_idxs)
fill_missing_similarity(data_test, test_pairwise_similarities, "timestamp_post", test_nan_timestamp_idxs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [20]:
fill_missing_median(data_train, "num_like_post")
fill_missing_median(data_train, "num_share_post")
fill_missing_median(data_train, "num_comment_post")

fill_missing_median(data_val, "num_like_post")
fill_missing_median(data_val, "num_share_post")
fill_missing_median(data_val, "num_comment_post")

fill_missing_median(data_test, "num_like_post")
fill_missing_median(data_test, "num_share_post")
fill_missing_median(data_test, "num_comment_post")

In [21]:
def create_additional_features(df):
  df['year'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.year
  df['month'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.month
  df['day'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.day
  df['weekday'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.weekday
  df['hour'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.hour

In [22]:
create_additional_features(data_train)
create_additional_features(data_val)
create_additional_features(data_test)

In [23]:
data_train = data_train.drop(columns=["timestamp_post"])
data_val = data_val.drop(columns=["timestamp_post"])
data_test = data_test.drop(columns=["timestamp_post"])

In [24]:
# Enrich information for training text by concating their features to text by words
train_datetime_statements = []
for (day, month, hour, weekday) in zip(data_train['day'], data_train['month'], data_train['hour'], data_train['weekday']):
  if weekday != 6:
    train_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    train_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  train_datetime_statements.append(train_pattern_post)

print(train_datetime_statements)

['Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ', 'Bài viết được đăng vào thứ 6, ngày 8 tháng 5 lúc 11 giờ', 'Bài viết được đăng vào thứ 7, ngày 6 tháng 6 lúc 1 giờ', 'Bài viết được đăng vào thứ 7, ngày 13 tháng 6 lúc 4 giờ', 'Bài viết được đăng vào thứ 2, ngày 9 tháng 3 lúc 7 giờ', 'Bài viết được đăng vào thứ 3, ngày 26 tháng 5 lúc 4 giờ', 'Bài viết được đăng vào thứ 4, ngày 13 tháng 5 lúc 19 giờ', 'Bài viết được đăng vào thứ 5, ngày 28 tháng 5 lúc 1 giờ', 'Bài viết được đăng vào thứ 4, ngày 15 tháng 4 lúc 11 giờ', 'Bài viết được đăng vào thứ 4, ngày 20 tháng 5 lúc 7 giờ', 'Bài viết được đăng vào thứ 5, ngày 21 tháng 5 lúc 5 giờ', 'Bài viết được đăng vào thứ 3, ngày 31 tháng 3 lúc 11 giờ', 'Bài viết được đăng vào thứ 2, ngày 13 tháng 4 lúc 1 giờ', 'Bài viết được đăng vào thứ 5, ngày 20 tháng 2 lúc 11 giờ', 'Bài viết được đăng vào thứ 6, ngày 26 tháng 6 lúc 8 giờ', 'Bài viết được đăng vào thứ 3, ngày 28 tháng 4 lúc 5 giờ', 'Bài viết được đăng vào thứ 4, ngày 29 tháng 4 lúc 0 

In [25]:
# Enrich information for validation text by concating their features to text by words
val_datetime_statements = []
for (day, month, hour, weekday) in zip(data_val['day'], data_val['month'], data_val['hour'], data_val['weekday']):
  if weekday != 6:
    val_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    val_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  val_datetime_statements.append(val_pattern_post)

In [26]:
test_datetime_statements = []
for (day, month, hour, weekday) in zip(data_test['day'], data_test['month'], data_test['hour'], data_test['weekday']):
  if weekday != 6:
    test_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    test_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  test_datetime_statements.append(test_pattern_post)

print(train_datetime_statements)

['Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ', 'Bài viết được đăng vào thứ 6, ngày 8 tháng 5 lúc 11 giờ', 'Bài viết được đăng vào thứ 7, ngày 6 tháng 6 lúc 1 giờ', 'Bài viết được đăng vào thứ 7, ngày 13 tháng 6 lúc 4 giờ', 'Bài viết được đăng vào thứ 2, ngày 9 tháng 3 lúc 7 giờ', 'Bài viết được đăng vào thứ 3, ngày 26 tháng 5 lúc 4 giờ', 'Bài viết được đăng vào thứ 4, ngày 13 tháng 5 lúc 19 giờ', 'Bài viết được đăng vào thứ 5, ngày 28 tháng 5 lúc 1 giờ', 'Bài viết được đăng vào thứ 4, ngày 15 tháng 4 lúc 11 giờ', 'Bài viết được đăng vào thứ 4, ngày 20 tháng 5 lúc 7 giờ', 'Bài viết được đăng vào thứ 5, ngày 21 tháng 5 lúc 5 giờ', 'Bài viết được đăng vào thứ 3, ngày 31 tháng 3 lúc 11 giờ', 'Bài viết được đăng vào thứ 2, ngày 13 tháng 4 lúc 1 giờ', 'Bài viết được đăng vào thứ 5, ngày 20 tháng 2 lúc 11 giờ', 'Bài viết được đăng vào thứ 6, ngày 26 tháng 6 lúc 8 giờ', 'Bài viết được đăng vào thứ 3, ngày 28 tháng 4 lúc 5 giờ', 'Bài viết được đăng vào thứ 4, ngày 29 tháng 4 lúc 0 

In [27]:
# Enrich information for training text by concating their numeric features to text by words
train_num_statements = []
for (num_like, num_share, num_comment) in zip(data_train['num_like_post'], data_train['num_share_post'], data_train['num_comment_post']):
  train_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  train_num_statements.append(train_num_post)

In [28]:
# Enrich information for validation text by concating their numeric features to text by words
val_num_statements = []
for (num_like, num_share, num_comment) in zip(data_val['num_like_post'], data_val['num_share_post'], data_val['num_comment_post']):
  val_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  val_num_statements.append(val_num_post)

In [29]:
# Enrich information for testing text by concating their numeric features to text by words
test_num_statements = []
for (num_like, num_share, num_comment) in zip(data_test['num_like_post'], data_test['num_share_post'], data_test['num_comment_post']):
  test_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  test_num_statements.append(test_num_post)

In [30]:
train_modified_texts = []
for (train_num_post, train_datetime_post, tr_sent) in zip(train_num_statements, train_datetime_statements, train_sents):
  train_text = train_num_post + ". " + train_datetime_post + ". " + tr_sent
  train_modified_texts.append(train_text)

In [31]:
val_modified_texts = []
for (val_num_post, val_datetime_post, val_sent) in zip(val_num_statements, val_datetime_statements, val_sents):
  val_text = val_num_post + ". " + val_datetime_post  + ". " + val_sent
  val_modified_texts.append(val_text)

In [32]:
test_modified_texts = []
for (test_num_post, test_datetime_post, test_sent) in zip(test_num_statements, test_datetime_statements, test_sents):
  test_text = test_num_post + ". " + test_datetime_post + ". " + test_sent
  test_modified_texts.append(test_text)

In [33]:
def convert_tokens_to_ids(texts, tokenizer, max_seq_length=256, labels=None):
    input_ids, attention_masks = [], []
    for text in texts:
        inputs = tokenizer.encode_plus(text, padding='max_length', max_length=max_seq_length, truncation=True)
        input_ids.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])

    if labels is not None:
        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(attention_masks, dtype=torch.long), torch.tensor(
            labels, dtype=torch.long)
    return torch.tensor(input_ids, dtype=torch.long), torch.tensor(attention_masks, dtype=torch.long)

In [34]:
def eval(val_loader, model, epoch, device):
    # Evaluate model
    model.eval()
    y_val = []
    val_preds = None
    print(f"EPOCH {epoch + 1}: ===EVALUATION===")
    for (input_ids, attention_mask, y_batch) in val_loader:
        y_pred = model(input_ids.to(device),
                       attention_mask=attention_mask.to(device))
        y_pred = y_pred.squeeze().detach().cpu().numpy()
        val_preds = np.atleast_1d(y_pred) if val_preds is None else np.concatenate(
            [val_preds, np.atleast_1d(y_pred)])
        y_val.extend(y_batch.tolist())

    val_preds = sigmoid(val_preds)
    score = f1_score(y_val, val_preds > 0.5, pos_label=0)
    roc_score = roc_auc_score(y_val, val_preds)
    print(f"PREDICT {sum(val_preds <= 0.5)} INFORMATIVES")
    print(f"ACTUALY {len(y_val) - sum(y_val)} INFORMATIVES")

    print(
        f"\n----- F1 score @0.5 = {score:.4f}\nROC-AUC Score = {roc_score:.4f}")
    return roc_score

In [35]:
def predict(test_df, model, config, tweet_tokenizer, vncorenlp, model_tokenizer, test_modified_texts):
    test_post_ids = test_df.id.to_list()

    test_ids, test_masks = convert_tokens_to_ids(test_modified_texts, model_tokenizer, 256)

    test_dataset = TensorDataset(test_ids, test_masks)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model.eval()
    test_preds = None
    for i, (input_ids, masks) in enumerate(test_dataloader):
        if i % 20 == 0 or i == len(test_dataloader):
            print(f"Predicted {i} posts.")
        y_pred = model(input_ids.cuda(), attention_mask=masks.cuda())
        y_pred = y_pred.squeeze().detach().cpu().numpy()
        test_preds = np.atleast_1d(y_pred) if test_preds is None else np.concatenate([test_preds, np.atleast_1d(y_pred)])

    test_preds = sigmoid(test_preds)
    test_preds = test_preds.tolist()
    final_result = []
    for post_id, test_pred in zip(test_post_ids, test_preds):
      final_result.append([post_id, test_pred])
    
    result_df = DataFrame(final_result)
    result_df.to_csv('/content/drive/MyDrive/VLSP-Fake-News-Detection/results.csv', index=False)

In [36]:
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [37]:
def save_checkpoint(model, tokenizer, checkpoint_path, epoch='best'):
    torch.save(model.state_dict(), os.path.join(
        checkpoint_path, f'model_{epoch}.bin'))
    model.config.to_json_file(os.path.join(checkpoint_path, 'config.json'))
    tokenizer.save_vocabulary(checkpoint_path)

In [38]:
EPOCHS = 4
BATCH_SIZE = 32
ACCUMULATION_STEPS = 5
LEARNING_RATE = 1e-5

config_path = '/content/drive/MyDrive/VLSP-Fake-News-Detection/config/electra_1.json'
single_model_config = json.load(open(config_path, 'r'))

In [39]:
seed = 7370
seed_everything(seed)

In [42]:
if single_model_config['model_type'] == 'BERT':
  print("===Use BERT model===")
  checkpoint_dir = '/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/vbert_base/'
  tokenizer = BertTokenizer.from_pretrained(
      single_model_config['model_name'], do_lower_case=False)
  tokenizer.add_tokens(['<url>'])
  config = BertConfig.from_pretrained(single_model_config['model_name'], num_labels=1,
                                      output_hidden_states=True)
  model = BertReINTELClassification.from_pretrained(
      single_model_config['model_name'], config=config)
  model.to(device)
  tsfm = model.bert
elif single_model_config['model_type'] == 'ROBERTA':
  print("===Use PhoBERT model===")
  checkpoint_dir = '/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/phobert_base/'
  tokenizer = AutoTokenizer.from_pretrained(
      single_model_config['model_name'])
  tokenizer.add_tokens(['<url>'])
  config = RobertaConfig.from_pretrained(single_model_config['model_name'], num_labels=1,
                                          output_hidden_states=True)
  model = RobertaReINTELClassification.from_pretrained(
      single_model_config['model_name'], config=config)
  # model.resize_token_embeddings(len(tokenizer))
  model.to(device)
  tsfm = model.roberta
elif single_model_config['model_type'] == 'ELECTRA':
  print("===Use ELECTRA model===")
  checkpoint_dir = '/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/'
  tokenizer = ElectraTokenizer.from_pretrained(
      single_model_config['model_name'], do_lower_case=False)
  tokenizer.add_tokens(['<url>'])
  config = ElectraConfig.from_pretrained(single_model_config['model_name'], num_labels=1,
                                          output_hidden_states=True, output_attentions=False)
  model = ElectraReINTELClassification.from_pretrained(
      single_model_config['model_name'], config=config)
  model.resize_token_embeddings(len(tokenizer))
  model.to(device)
  tsfm = model.electra
elif single_model_config['model_type'] == 'BERT4NEWS':
  print("===Use BERT4news model===")
  checkpoint_dir = '/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/bert4news/'
  tokenizer = BertTokenizer.from_pretrained(
      single_model_config['model_name'], do_lower_case=False)
  tokenizer.add_tokens(['<url>'])
  config = BertConfig.from_pretrained(single_model_config['model_name'], num_labels=1,
                                          output_hidden_states=True)
  model = Bert4newsReINTELClassification.from_pretrained(
      single_model_config['model_name'], config=config)
  #model.resize_token_embeddings(len(tokenizer))
  model.to(device)
  tsfm = model.bert4news
else:
  print("Model type invalid!!!")

print(f"Seed number: {seed}")

===Use ELECTRA model===


Some weights of the model checkpoint at FPTAI/velectra-base-discriminator-cased were not used when initializing ElectraReINTELClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraReINTELClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraReINTELClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraReINTELClassification were not initialized from the model checkpoint at FPTAI/velectra-base-discriminator-cased and are newly initialized: ['ln.weight', 'ln.

Seed number: 7370


In [43]:
tr_labels = data_train.label.to_list()
train_ids, train_masks, train_labels = convert_tokens_to_ids(train_modified_texts, tokenizer, 256, tr_labels)
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
train_sampler = SequentialSampler(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)

In [44]:
vl_labels = data_val.label.to_list()
val_ids, val_masks, val_labels = convert_tokens_to_ids(val_modified_texts, tokenizer, 256, vl_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)
val_sampler = SequentialSampler(val_dataset)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [45]:
class KimCNN(nn.Module):
    def __init__(self, embed_num, embed_dim, dropout_rate=0.1, kernel_num=3, kernel_sizes=[2,3,4], num_labels=2):
        super().__init__()
        self.num_labels = num_labels
        self.embed_num = embed_num
        self.embed_dim = embed_dim
        self.dropout = dropout_rate
        self.kernel_num = kernel_num
        self.kernel_sizes = kernel_sizes
        self.embed = nn.Embedding(self.embed_num, self.embed_dim)
        self.convs = nn.ModuleList([nn.Conv2d(1, self.kernel_num, (k, self.embed_dim)) for k in self.kernel_sizes])
        self.dropout = nn.Dropout(self.dropout)
        self.classifier = nn.Linear(len(self.kernel_sizes)*self.kernel_num, self.num_labels)
        
    def forward(self, inputs, labels=None):
        output = inputs.unsqueeze(1)
        output = [nn.functional.relu(conv(output)).squeeze(3) for conv in self.convs]
        output = [nn.functional.max_pool1d(i, i.size(2)).squeeze(2) for i in output]
        output = torch.cat(output, 1)
        output = self.dropout(output)
        logits = self.classifier(output)
        return logits

In [49]:
print(f"Base model: {model}")

Base model: ElectraReINTELClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32055, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,)

In [53]:
embed_num = 256
embed_dim = model.config.hidden_size 
dropout = model.config.hidden_dropout_prob
kernel_num = 3
kernel_sizes = [2,3,4]
num_labels = 1

cnn_model = KimCNN(embed_num, embed_dim, dropout_rate=dropout, kernel_num=kernel_num, kernel_sizes=kernel_sizes, num_labels=num_labels)
cnn_model.to(device)

KimCNN(
  (embed): Embedding(256, 768)
  (convs): ModuleList(
    (0): Conv2d(1, 3, kernel_size=(2, 768), stride=(1, 1))
    (1): Conv2d(1, 3, kernel_size=(3, 768), stride=(1, 1))
    (2): Conv2d(1, 3, kernel_size=(4, 768), stride=(1, 1))
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Linear(in_features=9, out_features=1, bias=True)
)

In [54]:
num_train_optimization_steps = int(EPOCHS * len(train_dataset) / BATCH_SIZE / ACCUMULATION_STEPS)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(
        np in n for np in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(
        np in n for np in no_decay)], 'weight_decay': 0.01}
]

optimizer = AdamW(optimizer_grouped_parameters,lr=LEARNING_RATE, correct_bias=False)
scheduler0 = get_constant_schedule(optimizer)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)

for child in tsfm.children():
  for param in child.parameters():
    param.require_grad = False

frozen = False
del scheduler0
torch.cuda.empty_cache()
gc.collect()
optimizer.zero_grad()

In [60]:
for epoch in range(EPOCHS):   
  print(f"Training on epoch {epoch + 1}")
  cnn_model.train()
  for step, batch in enumerate(train_loader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    with torch.no_grad():
        inputs,_ = model(input_ids, input_mask)
    loss = cnn_model(inputs, label_ids)
    loss = loss.mean()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  roc_score = eval(val_loader, model, epoch, "cuda")
  if roc_score >= best_score:
    best_score = roc_score
    save_checkpoint(model, tokenizer, checkpoint_dir, epoch=seed)
    print(f"Updated best score model!!! :{best_score}")
  print("==================Done=============")

Training on epoch 1


ValueError: ignored